In [ ]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.5.0

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [0]:
import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

In [0]:
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [0]:
train.to_csv('train.txt', sep='\t', index=False)
test.to_csv('test.txt', sep='\t', index=False)

In [0]:
dataset_train = nlp.data.TSVDataset("train.txt", field_indices = [1, 2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("test.txt", field_indices = [1, 2], num_discard_samples=1)

In [11]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [0]:
class BERTDataset(Dataset):
    
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i],))

    def __len__(self):
        return (len(self.labels))
    

In [0]:
max_len = 64
batch_size = 64
warmup_ratio = 0.02
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [0]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [0]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [0]:
class BERTClassifier(nn.Module):
    
    def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=None, params=None):
        
        super(BERTClassifier, self).__init__()
        
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size, num_classes)
        
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    
    def gen_attention_mask(self, token_ids, valid_length):
        
        attention_mask = torch.zeros_like(token_ids)
        
        for i, v in enumerate(valid_length):
            attention_mask[i][:v]=1
            
        return attention_mask.float()

    
    def forward(self, token_ids, valid_length, segment_ids):
        
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))
        
        if self.dr_rate:
            out = self.dropout(pooler)
            
        return self.classifier(out)
    

In [0]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [0]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.01}
]

In [0]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [0]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [0]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [0]:
def calc_accuracy(X, Y):
    
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
    
    return train_acc


In [0]:
for e in range(num_epochs):

    train_acc = 0.0
    test_acc = 0.0

    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):

        optimizer.zero_grad()

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()
        scheduler.step()
        
        train_acc += calc_accuracy(out, label)

        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e + 1, batch_id + 1, loss.data.cpu().numpy(), train_acc / (batch_id + 1)))

    print("epoch {} train acc {}".format(e + 1, train_acc / (batch_id + 1)))

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        
        valid_length = valid_length
        
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        
        test_acc += calc_accuracy(out, label)
        
    print("epoch {} test acc {}".format(e + 1, test_acc / (batch_id + 1)))
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


epoch 1 batch id 1 loss 0.692930281162262 train acc 0.53125
epoch 1 batch id 201 loss 0.7156891822814941 train acc 0.5268967661691543
epoch 1 batch id 401 loss 0.7080405950546265 train acc 0.5340554862842892
epoch 1 batch id 601 loss 0.7054370641708374 train acc 0.5375155990016639
epoch 1 batch id 801 loss 0.717943549156189 train acc 0.5377848002496879

epoch 1 train acc 0.5373795612956129


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.5448069852941176


...

epoch 27 batch id 1 loss 0.047116804867982864 train acc 0.984375
epoch 27 batch id 201 loss 0.0214090496301651 train acc 0.9803327114427861
epoch 27 batch id 401 loss 0.11256546527147293 train acc 0.9795043640897756
epoch 27 batch id 601 loss 0.008239291608333588 train acc 0.9808392262895175
epoch 27 batch id 801 loss 0.018261387944221497 train acc 0.9812148876404494

epoch 27 train acc 0.9812410311603116



epoch 27 test acc 0.6953890931372549


epoch 28 batch id 1 loss 0.1691398322582245 train acc 0.96875
epoch 28 batch id 201 loss 0.038298483937978745 train acc 0.9820429104477612
epoch 28 batch id 401 loss 0.12452568113803864 train acc 0.9818033042394015
epoch 28 batch id 601 loss 0.004071078263223171 train acc 0.9830230865224625
epoch 28 batch id 801 loss 0.09122873842716217 train acc 0.9831655742821473

epoch 28 train acc 0.9830104551045511



epoch 28 test acc 0.6933976715686274


epoch 29 batch id 1 loss 0.062094125896692276 train acc 0.984375
epoch 29 batch id 201 loss 0.026307323947548866 train acc 0.9828980099502488
epoch 29 batch id 401 loss 0.1942361444234848 train acc 0.9831281172069826
epoch 29 batch id 601 loss 0.007811684161424637 train acc 0.9833350665557404
